In [2]:
import cv2
import mediapipe as mp
import tensorflow as tf
import time
import numpy as np
import copy
import itertools
import torch


%load_ext autoreload
%autoreload 2

In [10]:
yolo = torch.hub.load('ultralytics/yolov5', 'yolov5n')

Using cache found in /Users/hugo/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-4-23 torch 1.11.0 CPU

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients
Adding AutoShape... 


In [11]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose

In [12]:
class KeyPointClassifier(object):
    """
    Classify hand keys points into 8 gestures
    
    Note: the classification model and class has been taken and refactored from https://github.com/kinivi/tello-gesture-control
    """
    def __init__(
        self,
        model_path="models/keypoint_classifier.tflite",
        
    ):
        self.interpreter = tf.lite.Interpreter(model_path=model_path)
        self.interpreter.allocate_tensors()
        self.input_details = self.interpreter.get_input_details()
        self.output_details = self.interpreter.get_output_details()

    def __call__(
        self,
        frame,
        hand_landmarks,
    ):
        # Landmark calculation
        landmark_list = self._calc_landmark_list(frame, hand_landmarks)

        # Conversion to relative coordinates / normalized coordinates
        pre_processed_landmark_list = self._pre_process_landmark(landmark_list)

        input_details_tensor_index = self.input_details[0]['index']
        self.interpreter.set_tensor(
            input_details_tensor_index,
            np.array([pre_processed_landmark_list], dtype=np.float32))
        self.interpreter.invoke()

        output_details_tensor_index = self.output_details[0]['index']

        result = self.interpreter.get_tensor(output_details_tensor_index)

        result_index = np.argmax(np.squeeze(result))

        return result_index
    
    def _pre_process_landmark(self, landmark_list):
        temp_landmark_list = copy.deepcopy(landmark_list)

        # Convert to relative coordinates
        base_x, base_y = 0, 0
        for index, landmark_point in enumerate(temp_landmark_list):
            if index == 0:
                base_x, base_y = landmark_point[0], landmark_point[1]

            temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
            temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

        # Convert to a one-dimensional list
        temp_landmark_list = list(
            itertools.chain.from_iterable(temp_landmark_list))

        # Normalization
        max_value = max(list(map(abs, temp_landmark_list)))

        def normalize_(n):
            return n / max_value

        temp_landmark_list = list(map(normalize_, temp_landmark_list))

        return temp_landmark_list
    
    def _calc_landmark_list(self, image, landmarks):
            image_width, image_height = image.shape[1], image.shape[0]

            landmark_point = []

            # Keypoint
            for _, landmark in enumerate(landmarks.landmark):
                landmark_x = min(int(landmark.x * image_width), image_width - 1)
                landmark_y = min(int(landmark.y * image_height), image_height - 1)
                # landmark_z = landmark.z

                landmark_point.append([landmark_x, landmark_y])

            return landmark_point

In [13]:
def calc_bounding_rect(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_array = np.empty((0, 2), int)

    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)

        landmark_point = [np.array((landmark_x, landmark_y))]

        landmark_array = np.append(landmark_array, landmark_point, axis=0)

    x, y, w, h = cv2.boundingRect(landmark_array)

    return [x, y, x + w, y + h]

def calc_center(brect):
    return (brect[0] + brect[2]) / 2, (brect[1] + brect[3]) / 2

def calc_person_center(p):
    return (int(p.xmin) + int(p.xmax)) / 2, (int(p.ymin) + int(p.ymax)) / 2

def euclidian_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

def draw_info(image, brect, hand_sign_text = ""):
    # Outer rectangle
    cv2.rectangle(image, (brect[0], brect[1]), (brect[2], brect[3]),
                    (0, 0, 0), 3)

    # Text
    cv2.rectangle(image, (brect[0], brect[1]), (brect[2], brect[1] - 22),
                     (0, 0, 0), -1)
    cv2.putText(image, hand_sign_text, (brect[0] + 5, brect[1] - 4),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)

def draw_person(image, p):
    cv2.rectangle(image, (int(p.xmin), int(p.ymin)), (int(p.xmax), int(p.ymax)), (0, 0, 255), 2)

In [24]:

# For webcam input
cap = cv2.VideoCapture(0)
key_point_classifier = KeyPointClassifier()
with mp_hands.Hands(
  model_complexity=0,
  min_detection_confidence=0.5,
  min_tracking_confidence=0.5,
  max_num_hands=4) as hands:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue
    
    image = cv2.flip(image, 1)

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)
    
    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    yolo_pred = yolo(image)
    persons = yolo_pred.pandas().xyxy[0]
    persons = persons[persons['class'] == 0]
    if results.multi_hand_landmarks:
      hand_center = (-1, -1)
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
          image,
          hand_landmarks,
          mp_hands.HAND_CONNECTIONS,
          mp_drawing_styles.get_default_hand_landmarks_style(),
          mp_drawing_styles.get_default_hand_connections_style())
        
        gesture_idx = key_point_classifier(image, hand_landmarks)
        if gesture_idx == 2:
          brect = calc_bounding_rect(image, hand_landmarks)
          draw_info(image, brect, "Sign detected")
          hand_center = calc_center(calc_bounding_rect(image, hand_landmarks))
        # gesture_idx = key_point_classifier(image, hand_landmarks)
        # print(gesture_idx)
      # for _, p in persons.iterrows():
      #   distances.append(euclidian_distance(calc_person_center(p), hand_center))
      #   draw_person(image, p)
      if hand_center != (-1, -1):
        distances = []
        for _, p in persons.iterrows():
          distances.append(euclidian_distance(calc_person_center(p), hand_center))
        if len(distances) > 0:
          min_idx = np.argmin(distances)
          draw_person(image, persons.iloc[min_idx])


    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Hands', image)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()

KeyboardInterrupt: 